In [ ]:
!pip install transformers

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.77-cp310-cp310-linux_x86_64.whl size=3695621 sha256=f1eaae5cf38acb53cc83bfa9f54316b2bbe4106b38694c752ffea0ad304dbd84
  Stored in directory: /root/.cache/pip/wheels/ed/55/a1/6d6c2ef6fed3ef054b4170d8bcd05a09e6dc971db7fad955ff
Successfully built llama-cpp-python


In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_name = "NousResearch/Hermes-2-Theta-Llama-3-8B-GGUF"
model_file = "Hermes-2-Pro-Llama-3-Instruct-Merged-DPO-Q4_K_M.gguf" # this is the specific model file we'll use in this example. It's a 4-bit quant, but other levels of quantization are available in the model repo if preferred
model_path = hf_hub_download(model_name, filename=model_file)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)-Llama-3-Instruct-Merged-DPO-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
llm = Llama(
    model_path=model_path,
    n_ctx=16000,  # Context length to use
    n_threads=32,  # Number of CPU threads to use
    n_gpu_layers=0  # Number of model layers to offload to GPU
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--NousResearch--Hermes-2-Theta-Llama-3-8B-GGUF/snapshots/ded6bdf5c60c7211eb8cb5cec6c91fa358b509b7/Hermes-2-Pro-Llama-3-Instruct-Merged-DPO-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-Instruct-Merged-DPO
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32           

In [ ]:
## Generation kwargs
generation_kwargs = {
    "max_tokens": 20000,
    "stop": ["</s>"],
    "echo": False,  # Echo the prompt in the output
    "top_k": 1  # This is essentially greedy decoding
}

In [ ]:
## Run inference
prompt = "The meaning of life is "
res = llm(prompt, **generation_kwargs)  # Res is a dictionary


llama_print_timings:        load time =    4564.19 ms
llama_print_timings:      sample time =    8772.92 ms /  3013 runs   (    2.91 ms per token,   343.44 tokens per second)
llama_print_timings: prompt eval time =    4563.71 ms /     7 tokens (  651.96 ms per token,     1.53 tokens per second)
llama_print_timings:        eval time = 2153127.82 ms /  3012 runs   (  714.85 ms per token,     1.40 tokens per second)
llama_print_timings:       total time = 2186921.44 ms /  3019 tokens


In [ ]:
print(res["choices"][0]["text"])

42, or so says Douglas Adams in his science fiction series "The Hitchhiker's Guide to the Galaxy." But what does that really mean? Is it just a humorous way to answer a profound question, or is there something deeper at play?
In this episode of Philosophy Bites, David Edmonds and Nigel Warburton discuss the meaning of life with philosopher Simon Blackburn. They explore whether there can be a single answer to the question, and whether it's even possible for us to know what that answer might be.
Blackburn suggests that the search for the meaning of life is often driven by our fear of death, and that we may never find a definitive answer because the concept of "meaning" itself is subjective. He also argues that there are many different ways to live a meaningful life, depending on one's values and beliefs.
Ultimately, Blackburn concludes that while we may not be able to definitively answer the question of what the meaning of life is, we can still strive to create our own personal meanings 

In [ ]:
!pip install pandas scikit-learn

In [ ]:
import pandas as pd

import pandas as pd

## Load your TSV dataset
df = pd.read_csv('merged_reviews_finalDS.tsv', delimiter='\t')  # Replace with your dataset path
reviews = df['text'].tolist()
labels = df['label'].tolist()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Function to classify review using the model
def classify_review(review, model, generation_kwargs):
    prompt = f"Classify the sentiment of this review as positive, neutral, or negative: '{review}'"
    response = model(prompt, **generation_kwargs)
    text = response["choices"][0]["text"].lower()
    if 'positive' in text:
        return 1
    elif 'negative' in text:
        return -1
    elif 'neutral' in text:
        return 0
    else:
        return None  # In case the response is ambiguous


In [ ]:

## Generate predictions for all reviews
predictions = [classify_review(review, llm, generation_kwargs) for review in reviews]

Llama.generate: prefix-match hit


In [ ]:
## Filter out None values (ambiguous classifications)
filtered_reviews = [reviews[i] for i in range(len(predictions)) if predictions[i] is not None]
filtered_labels = [labels[i] for i in range(len(predictions)) if predictions[i] is not None]
filtered_predictions = [predictions[i] for i in range(len(predictions)) if predictions[i] is not None]


In [ ]:
## Calculate accuracy
accuracy = accuracy_score(filtered_labels, filtered_predictions)

## Generate confusion matrix
conf_matrix = confusion_matrix(filtered_labels, filtered_predictions)

## Generate classification report
class_report = classification_report(filtered_labels, filtered_predictions, target_names=['negative', 'neutral', 'positive'])

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")
